# Microlens-Submit on the Roman Research Nexus

This tutorial demonstrates how to use `microlens-submit` within the Roman Research Nexus environment.

In [ ]:
!pip install microlens-submit

In [ ]:
!microlens-submit nexus-init --team-name "My Nexus Team" --tier "advanced"

In [ ]:
from microlens_submit import load
submission = load(".")
print(submission.hardware_info)

Add solutions and export the project using the CLI.

In [ ]:
!microlens-submit add-solution my-event binary_lens --param t0=123.4 --param u0=0.1

In [ ]:
!microlens-submit export submission.zip --force